Name: Ozan Bulut

I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution to 16.3-5 and 18
[Your solutions come here]

## Pr?gr?mm?ng?H?m?w?rk 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [81]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [126]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}
letterInv = {letter2idx[i]:i for i in letter2idx.keys()}


T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


# Part 1
Because the distribution is markovian, we only need to consider previous letter.



In [131]:
import numpy as np
T=np.array(T).astype(np.float)

def trivial_case(x0):
    normalised = T[letter2idx[x0]]/np.sum(T[letter2idx[x0]][:])
    return np.random.choice(alphabet,p=normalised)

seq = "."
for i in range(1,15):
    seq += trivial_case(seq[-1])
print (seq[1:])

m.re.ngd.se.ts


# Part 2

In this part, I've modelled the situation where middle of the string is missing. Let's say we know start charecter $X_0$ and last character $X_{N+1}$. Middle of the string $X_{1:N}$ is modelled in this form:
$$p(X_{1:N}|X_0=x_0,X_{N+1}=x_{N+1}) = \frac{p(X_{1:N},X_0=x_0,X_{N+1}=x_{N+1})}{p(X_0=x_0,X_{N+1}=x_{N+1})} = 
\frac{{p(X_{1:N},X_{N+1}=x_{N+1},X_0=x_0)} }{ \sum_{1:N}{p(x_{0:N+1})}} $$ 
because the model is Markov(1) using following equation
$$ 
p(X_{0:N+1}) = p(X_0)\prod_1^{N+1}{p(X_i|X_{i-1})} 
$$
we can derive
$$ 
p(X_{1:N}|X_0=x_0,X_{N+1}=x_{N+1}) = \frac{p(X_0=x_0)p(X_1|X_0=x_0)\prod_2^{N}{p(X_i|X_{i-1})}p(X_{N+1}=x_{N+1}|X_N) }{p(X_0=x_0) p(X_{N+1}=x_{N+1})}=
\frac{p(X_1|X_0=x_0)p(X_{N+1}=x_{N+1}|X_N)\prod_2^{N}{p(X_i|X_{i-1})}}{p(X_{N+1}=x_{N+1})}
$$


In [185]:
power_list = [T] # pre compute powers of the matrix
for i in range(1,20):
    power_list.append(np.multiply(power_list[-1],T))

def nontrivial_case(N,x0,xN):
    power_matrix = power_list[N]
    last_matrix = (T[:][letter2idx[xN]])
    matrix = np.multiply(power_matrix,last_matrix)
    normalised = matrix[letter2idx[x0]][:]/np.sum(matrix[letter2idx[x0]][:])
    return np.random.choice(alphabet,p=normalised)

def generate(st):
    str = list("."+st)
    left=1
    right=1
    while left<len(str):
        if(str[left]=='_'):
            if  right<left:
                right=left
            while right<len(str) and str[right]=='_':
                right+=1
            if right<len(str):
                str[left]=nontrivial_case(right-left,str[left-1],str[right])
            else:
                str[left] = trivial_case(str[left-1])
        left+=1
    return ''.join(str[1:])



In [186]:
for i in test_strings:
    for j in range(1,6):
        print(generate(i))

therbre.n.f.x.
therbre.n.f.x.
therbr.an.f.x.
therbr.tn.fex.
the.br.tn.f.x.
tu.st.tnt.toube.answiram
tutst.tn..toube.snswero.
ru.st.ant.toube..nswerpe
autst.an..toube.tnsw.rg.
tutst.tn..toube.insweres
in.ato.ta.h.n..ce.rrisg
in.ath.caihend.terroing
intat..taihon...e.rei.g
it.at..tanhent.ae.rei.g
insath.taihen..seer.ing
quret.ouaz.the.ht.as.at..th.
quret.a.az.the.ht..w.and.at.
qur.t.aloz.an.s.t.th.tha.t..
qutht.alez.an.a.t.t..t.a.an.
qur.t.ie.z.the.ot.th.tha.th.


# Part 3

In [184]:
def arg_nontrivial_case(N,x0,xN):
    power_matrix = power_list[N]
    last_matrix = (T[:][letter2idx[xN]])
    matrix = np.multiply(power_matrix,last_matrix)
    normalised = matrix[letter2idx[x0]][:]/np.sum(matrix[letter2idx[x0]][:])
    return np.argmax(normalised)

def arg_trivial_case(x0):
    normalised = T[letter2idx[x0]]/np.sum(T[letter2idx[x0]][:])
    return np.argmax(normalised)

def generate_arg(st):
    str = list("."+st)
    left=1
    right=1
    while left<len(str):
        if(str[left]=='_'):
            if  right<left:
                right=left
            while right<len(str) and str[right]=='_':
                right+=1
            if right<len(str):
                str[left]=letterInv[arg_nontrivial_case(right-left,str[left-1],str[right])]
            else:
                str[left] = letterInv[arg_trivial_case(str[left-1])]
        left+=1
    return ''.join(str[1:])



In [179]:
for i in test_strings:
    print(generate_arg(i))

therbr.tn.f.x.
tutst.tn..toube.tnswere.
in.ath.ta.hen...e.r.i.g
qur.t.a.az.the.ht.th.the.th.


# Part 4
Considering white spaces and other charecters we could implement better estimator.